In [4]:
class CheckersGame:
    def __init__(self):
        self.board = self.create_board()
        self.current_player = 'black'

    def create_board(self):
        # Initialize the board with empty spaces and pieces
        board = [
            ['_', 'b', '_', 'b', '_', 'b', '_', 'b'],
            ['b', '_', 'b', '_', 'b', '_', 'b', '_'],
            ['_', 'b', '_', 'b', '_', 'b', '_', 'b'],
            ['_', '_', '_', '_', '_', '_', '_', '_'],
            ['_', '_', '_', '_', '_', '_', '_', '_'],
            ['r', '_', 'r', '_', 'r', '_', 'r', '_'],
            ['_', 'r', '_', 'r', '_', 'r', '_', 'r'],
            ['r', '_', 'r', '_', 'r', '_', 'r', '_'],
        ]
        return board

    def is_valid_move(self, start, end, player):
        x1, y1 = start
        x2, y2 = end

        if not (0 <= x2 < 8 and 0 <= y2 < 8):  # Check board boundaries
            return False
        if self.board[x2][y2] != '_':  # Destination must be empty
            return False

        direction = 1 if player == 'black' else -1

        # Normal move
        if abs(x2 - x1) == 1 and abs(y2 - y1) == 1:
            if (x2 - x1) == direction:
                return True

        # Capture move
        if abs(x2 - x1) == 2 and abs(y2 - y1) == 2:
            mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
            if self.board[mid_x][mid_y].lower() not in ('_', player):
                if (x2 - x1) == 2 * direction:
                    return True

        return False

    def make_move(self, start, end):
        x1, y1 = start
        x2, y2 = end

        self.board[x2][y2] = self.board[x1][y1]
        self.board[x1][y1] = '_'

        # Check for capture
        if abs(x2 - x1) == 2:
            mid_x, mid_y = (x1 + x2) // 2, (y1 + y2) // 2
            self.board[mid_x][mid_y] = '_'

        # Promote to king
        if x2 == 0 and self.board[x2][y2] == 'r':
            self.board[x2][y2] = 'R'
        if x2 == 7 and self.board[x2][y2] == 'b':
            self.board[x2][y2] = 'B'

    def get_valid_moves(self, player):
        moves = []
        for i in range(8):
            for j in range(8):
                if self.board[i][j].lower() == player:
                    for dx, dy in [(1, 1), (1, -1), (-1, 1), (-1, -1)]:
                        x, y = i + dx, j + dy
                        if self.is_valid_move((i, j), (x, y), player):
                            moves.append(((i, j), (x, y)))
        return moves

    def game_over(self):
        return not self.get_valid_moves('black') or not self.get_valid_moves('red')

    def evaluate_board(self):
        # Evaluate board: positive for black, negative for red
        score = 0
        for row in self.board:
            for cell in row:
                if cell == 'b':
                    score += 1
                elif cell == 'B':  # King
                    score += 3
                elif cell == 'r':
                    score -= 1
                elif cell == 'R':  # King
                    score -= 3
        return score

    def alpha_beta_pruning(self, depth, alpha, beta, maximizing_player):
        if depth == 0 or self.game_over():
            return self.evaluate_board()

        if maximizing_player:
            max_eval = float('-inf')
            for move in self.get_valid_moves('black'):
                start, end = move
                self.make_move(start, end)
                eval = self.alpha_beta_pruning(depth - 1, alpha, beta, False)
                self.make_move(end, start)  # Undo move
                max_eval = max(max_eval, eval)
                alpha = max(alpha, eval)
                if beta <= alpha:
                    break
            return max_eval
        else:
            min_eval = float('inf')
            for move in self.get_valid_moves('red'):
                start, end = move
                self.make_move(start, end)
                eval = self.alpha_beta_pruning(depth - 1, alpha, beta, True)
                self.make_move(end, start)  # Undo move
                min_eval = min(min_eval, eval)
                beta = min(beta, eval)
                if beta <= alpha:
                    break
            return min_eval

    def print_board(self):
        for row in self.board:
            print(' '.join(row))
        print()

    def play(self):
        while not self.game_over():
            self.print_board()
            if self.current_player == 'black':
                _, best_move = max([(self.alpha_beta_pruning(3, float('-inf'), float('inf'), True), move) for move in self.get_valid_moves('black')])
                self.make_move(*best_move)
                self.current_player = 'red'
            else:
                _, best_move = min([(self.alpha_beta_pruning(3, float('-inf'), float('inf'), False), move) for move in self.get_valid_moves('red')])
                self.make_move(*best_move)
                self.current_player = 'black'

        self.print_board()
        if self.evaluate_board() > 0:
            print("Black wins!")
        elif self.evaluate_board() < 0:
            print("Red wins!")
        else:
            print("It's a draw!")


if __name__ == '__main__':
    game = CheckersGame()
    game.play()


_ b _ b _ b _ b
b _ b _ b _ b _
_ b _ b _ b _ b
_ _ _ _ _ _ _ _
_ _ _ _ _ _ _ _
r _ r _ r _ r _
_ r _ r _ r _ r
r _ r _ r _ r _

It's a draw!
